In [0]:
%sql
Select * from analytics.bronze.crm_prd_info

In [0]:
%sql
USE CATALOG analytics;
USE SCHEMA silver;


## Read Bronze table

In [0]:
df = spark.table("bronze.crm_prd_info")
df.display()

## Remove duplicate products

In [0]:
df = df.dropDuplicates(["prd_id"])
df.display()

## Trim all string columns

In [0]:
from pyspark.sql.functions import trim, col
from pyspark.sql.types import StringType

for f in df.schema.fields:
    if isinstance(f.dataType, StringType):
        df = df.withColumn(f.name, trim(col(f.name)))

df.display()

## Handle missing product cost

In [0]:
df = df.fillna({"prd_cost": 0})
df.display()

In [0]:
%sql
select distinct (prd_line) from analytics.bronze.crm_prd_info

## Normalize product line

In [0]:
from pyspark.sql.functions import when, col

df = df.withColumn(
    "prd_line",
    when(col("prd_line") == "R", "Road")
    .when(col("prd_line") == "M", "Mountain")
    .when(col("prd_line") == "S", "Sport")
    .when(col("prd_line") == "T", "Touring")
    .otherwise("Unknown")
)

df.display()

## Rename columns to friendly names

In [0]:
df = (
    df
    .withColumnRenamed("prd_id", "product_id")
    .withColumnRenamed("prd_key", "product_key")
    .withColumnRenamed("prd_nm", "product_name")
    .withColumnRenamed("prd_cost", "product_cost")
    .withColumnRenamed("prd_line", "product_line")
    .withColumnRenamed("prd_start_dt", "start_date")
    .withColumnRenamed("prd_end_dt", "end_date")
)

df.display()

In [0]:
df.select("product_line").distinct().display()
df.select("product_cost").summary().display()


## Write Silver table

In [0]:
df.write \
  .mode("overwrite") \
  .format("delta") \
  .saveAsTable("silver.crm_products")


In [0]:
spark.table("silver.crm_products").display()